# Tensorflow MNIST Model

 * Wrap a Tensorflow MNIST python model for use as a prediction microservice in seldon-core
 
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install seldon-core
```

## Train locally
 

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
import tensorflow as tf

if __name__ == '__main__':
    
    x = tf.placeholder(tf.float32, [None,784], name="x")

    W = tf.Variable(tf.zeros([784,10]))
    b = tf.Variable(tf.zeros([10]))

    y = tf.nn.softmax(tf.matmul(x,W) + b, name="y")

    y_ = tf.placeholder(tf.float32, [None, 10])


    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    init = tf.initialize_all_variables()

    sess = tf.Session()
    sess.run(init)

    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict = {x: mnist.test.images, y_:mnist.test.labels}))

    saver = tf.train.Saver()

    saver.save(sess, "model/deep_mnist_model")



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.9109


Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-python37:0.13 deep-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [3]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 deep-mnist:0.1

223e9360b237670a25ccfcec4a592d16bf4f6c9fc392ddcb79d22ed1c5df499f


Send some random features that conform to the contract

In [5]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[0.075 0.186 0.789 0.717 0.236 0.553 0.514 0.527 0.59  0.267 0.826 0.947
  0.629 0.211 0.334 0.341 0.138 0.879 0.522 0.326 0.838 0.703 0.455 0.718
  0.507 0.691 0.76  0.794 0.829 0.835 0.435 0.79  0.215 0.646 0.227 0.605
  0.204 0.214 0.814 0.296 0.799 0.225 0.598 0.301 0.624 0.213 0.672 0.895
  0.042 0.534 0.146 0.018 0.362 0.191 0.162 0.107 0.287 0.673 0.938 0.771
  0.457 0.696 0.82  0.911 0.629 0.772 0.267 0.719 0.904 0.536 0.98  0.494
  0.273 0.022 0.679 0.599 0.417 0.645 0.757 0.254 0.853 0.696 0.083 0.056
  0.312 0.17  0.021 0.438 0.625 0.393 0.597 0.127 0.624 0.004 0.507 0.976
  0.853 0.245 0.735 0.837 0.524 0.926 0.533 0.207 0.568 0.824 0.834 0.868
  0.941 0.56  0.698 0.419 0.817 0.953 0.549 0.278 0.637 0.436 0.81  0.359
  0.775 0.663 0.03  0.106 0.717 0.668 0.931 0.295 0.784 0.393 0.879 0.599
  0.457 0.467 0.944 0.379 0.107 0.31  0.343 0.642 0.763 0.831 0.198 0.072
  0.492 0.34  0.035 0.744 0.773 0.027 0.575 0.729

In [6]:
!docker rm mnist_predictor --force

mnist_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [6]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

## Wrap Model and Test

In [14]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python37:0.13 deep-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [15]:
!kubectl create -f deep_mnist.json

seldondeployment.machinelearning.seldon.io/deep-mnist created


In [16]:
!kubectl rollout status deploy/deep-mnist-single-model-8969cc0

Waiting for deployment "deep-mnist-single-model-8969cc0" rollout to finish: 0 of 1 updated replicas are available...
deployment "deep-mnist-single-model-8969cc0" successfully rolled out


In [17]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    deep-mnist --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[0.891 0.499 0.792 0.386 0.739 0.092 0.986 0.789 0.758 0.109 0.267 0.834
  0.542 0.025 0.02  0.16  0.6   0.324 0.71  0.515 0.346 0.943 0.109 0.455
  0.243 0.023 0.901 0.465 0.249 0.442 0.8   0.875 0.772 0.588 0.995 0.578
  0.254 0.461 0.885 0.068 0.157 0.489 0.486 0.951 0.123 0.052 0.72  0.515
  0.002 0.122 0.035 0.04  0.368 0.373 0.447 0.452 0.344 0.323 0.673 0.145
  0.206 0.721 0.749 0.759 0.184 0.86  0.061 0.304 0.429 0.621 0.723 0.916
  0.334 0.452 0.883 0.391 0.861 0.686 0.846 0.316 0.987 0.853 0.231 0.06
  0.763 0.215 0.119 0.001 0.234 0.717 0.765 0.42  0.71  0.605 0.166 0.192
  0.726 0.133 0.785 0.307 0.7   0.187 0.153 0.704 0.1   0.255 0.155 0.555
  0.89  0.312 0.884 0.85  0.361 0.992 0.061 0.781 0.678 0.537 0.292 0.007
  0.951 0.46  0.585 0.338 0.552 0.751 0.842 0.31  0.343 0.149 0.712 0.011
  0.209 0.412 0.859 0.859 0.159 0.366 0.644 0.196 0.347 0.527 0.823 0.737
  0.341 0.258 0.605 0.441 0.982 0.765 0.037 0.278 

In [16]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
